### Installs

In [ ]:
!pip install bpemb
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 13.0 MB/s eta 0:00:00


### Imports

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import math
import matplotlib.pyplot as plt
import random
from bpemb import BPEmb
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.nn.utils.rnn import pack_padded_sequence
from torch.nn.utils.rnn import pad_packed_sequence
from torch.optim.lr_scheduler import ExponentialLR, CyclicLR
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from datasets import load_dataset
from tqdm import tqdm
from typing import List, Tuple


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
random.seed(seed)
np.random.seed(seed)

### Data, and embeddings

In [ ]:
dataset = load_dataset("copenlu/answerable_tydiqa")

train_df = dataset['train'].to_pandas()
train_df = train_df[train_df['language'].isin(['indonesian', 'arabic', 'bengali'])]

val_df = dataset['validation'].to_pandas()
val_df = val_df[val_df['language'].isin(['indonesian', 'arabic', 'bengali'])]

train_df['concat_text'] = '<q> ' + train_df['question_text'] + ' </q> <d> ' + train_df['document_plaintext'] + ' </d>'
val_df['concat_text'] = '<q> ' + val_df['question_text'] + ' </q> <d> ' + val_df['document_plaintext'] + ' </d>'

train_df['is_answerable'] = train_df['annotations'].apply(lambda x: int(x.get('answer_start', [-1])[0] != -1))
val_df['is_answerable'] = val_df['annotations'].apply(lambda x: int(x.get('answer_start', [-1])[0] != -1))

# divide data set
train_arab = train_df[train_df['language'] == 'arabic'][['concat_text','is_answerable']]
train_indo = train_df[train_df['language'] == 'indonesian'][['concat_text','is_answerable']]
train_beng = train_df[train_df['language'] == 'bengali'][['concat_text','is_answerable']]

val_arab = val_df[val_df['language'] == 'arabic'][['concat_text','is_answerable']]
val_indo = val_df[val_df['language'] == 'indonesian'][['concat_text','is_answerable']]
val_beng = val_df[val_df['language'] == 'bengali'][['concat_text','is_answerable']]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/116067 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13325 [00:00<?, ? examples/s]

In [ ]:
vocab_size = 10000
embedding_dim = 50

#bpemb_en = BPEmb(lang="en", dim=embedding_dim, vs=vocab_size, add_pad_emb=True) # English (for testing)
bpemb_bn = BPEmb(lang="bn", dim=embedding_dim, vs=vocab_size) # Bengali
bpemb_ar = BPEmb(lang="ar", dim=embedding_dim, vs=vocab_size) # Arabic
bpemb_id = BPEmb(lang="id", dim=embedding_dim, vs=vocab_size) # indonesian
# Extract the embeddings and add an embedding for our extra [PAD] token
embeddings_ar = np.concatenate([bpemb_ar.emb.vectors, np.zeros(shape=(1,embedding_dim))], axis=0)
embeddings_bn = np.concatenate([bpemb_bn.emb.vectors, np.zeros(shape=(1,embedding_dim))], axis=0)
embeddings_id = np.concatenate([bpemb_id.emb.vectors, np.zeros(shape=(1,embedding_dim))], axis=0)
# Extract the vocab and add an extra [PAD] token
vocabulary_ar = bpemb_ar.emb.index_to_key + ['<pad>']
vocabulary_bn = bpemb_bn.emb.index_to_key + ['<pad>']
vocabulary_id = bpemb_id.emb.index_to_key + ['<pad>']

print(embeddings_ar.shape)
print(len(vocabulary_ar))

downloading https://nlp.h-its.org/bpemb/bn/bn.wiki.bpe.vs10000.model


100%|██████████| 471203/471203 [00:00<00:00, 654295.19B/s]


downloading https://nlp.h-its.org/bpemb/bn/bn.wiki.bpe.vs10000.d50.w2v.bin.tar.gz


100%|██████████| 1933584/1933584 [00:01<00:00, 1912308.49B/s]


downloading https://nlp.h-its.org/bpemb/ar/ar.wiki.bpe.vs10000.model


...

downloading https://nlp.h-its.org/bpemb/id/id.wiki.bpe.vs10000.model


100%|██████████| 396303/396303 [00:00<00:00, 692518.71B/s]


downloading https://nlp.h-its.org/bpemb/id/id.wiki.bpe.vs10000.d50.w2v.bin.tar.gz


100%|██████████| 1920574/1920574 [00:00<00:00, 2219778.94B/s]


(10001, 50)
10001


In [ ]:
def text_to_batch_bilstm(text: List, tokenizer, max_len=512) -> Tuple[List, List]:
    """
    Creates a tokenized batch for input to a bilstm model
    :param text: A list of sentences to tokenize
    :param tokenizer: A tokenization function to use (i.e. fasttext)
    :return: Tokenized text as well as the length of the input sequence
    """
    # Some light preprocessing
    input_ids = [tokenizer.encode_ids_with_eos(t)[:max_len] for t in text]

    return input_ids, [len(ids) for ids in input_ids]


def collate_batch_bilstm(input_data: Tuple) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    Combines multiple data samples into a single (1) batch
    :param input_data: The combined input_ids, seq_lens, and labels for the batch
    :return: A tuple of tensors (input_ids, seq_lens, labels)
    """
    input_ids = [i[0][0] for i in input_data]
    seq_lens = [i[1][0] for i in input_data]
    labels = [i[2] for i in input_data]

    max_length = max([len(i) for i in input_ids])

    # Pad all of the input samples to the max length (25000 is the ID of the [PAD] token)
    input_ids = [(i + [vocab_size] * (max_length - len(i))) for i in input_ids]

    # Make sure each sample is max_length long
    assert (all(len(i) == max_length for i in input_ids))
    return torch.tensor(input_ids), torch.tensor(seq_lens), torch.tensor(labels)


class ClassificationDatasetReader(Dataset):
  def __init__(self, df, tokenizer):
    """
    :param df: Dataframe
    :param tokenizer: bpemb entity
    """
    self.df = df
    self.tokenizer = tokenizer

  def __len__(self):
    return len(self.df)

  def __getitem__(self, idx):
    row = self.df.values[idx]
    # Calls the text_to_batch function
    input_ids,seq_lens = text_to_batch_bilstm([row[0]], self.tokenizer)
    label = row[1]
    return input_ids, seq_lens, label

### Define model

In [ ]:
class BiLSTM(nn.Module):

    def __init__(
            self,
            pretrained_embeddings: torch.tensor,
            lstm_dim: int,
            num_layers: int,
            dropout_prob: float = 0.1,
            n_classes: int = 2
    ):
        super(BiLSTM, self).__init__()

        self.model = nn.ModuleDict({
            'embeddings': nn.Embedding.from_pretrained(pretrained_embeddings, padding_idx=pretrained_embeddings.shape[0] - 1),
            'bilstm': nn.LSTM(
                pretrained_embeddings.shape[1],
                lstm_dim,
                num_layers,
                batch_first=True,
                dropout=dropout_prob,
                bidirectional=True),
            'cls': nn.Linear(2*lstm_dim, n_classes)
        })
        self.n_classes = n_classes
        self.dropout = nn.Dropout(p=dropout_prob)

        self._init_weights()

    def _init_weights(self):
        all_params = list(self.model['bilstm'].named_parameters()) + \
                     list(self.model['cls'].named_parameters())
        for n,p in all_params:
            if 'weight' in n:
                nn.init.xavier_normal_(p)
            elif 'bias' in n:
                nn.init.zeros_(p)

    def forward(self, inputs, input_lens, labels = None):

        embeds = self.model['embeddings'](inputs)

        lstm_in = nn.utils.rnn.pack_padded_sequence(
            embeds,
            input_lens.cpu(),
            batch_first=True,
            enforce_sorted=False
        )
        lstm_out, hidden = self.model['bilstm'](lstm_in)

        lstm_out,_ = nn.utils.rnn.pad_packed_sequence(lstm_out, batch_first=True)

        ff_in = self.dropout(torch.max(lstm_out, 1)[0])

        logits = self.model['cls'](ff_in).view(-1, self.n_classes)
        outputs = (logits,)
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)
            outputs = (loss,) + outputs

        return outputs

### Define Training and evaluation

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

def accuracy(preds, labels):
  return np.sum(preds == labels) / len(labels)

def to_numpy(torch_arr):
  return torch_arr.detach().cpu().numpy()

def evaluate(model: nn.Module, valid_dl: DataLoader):
  model.eval()
  labels_all = []
  preds_all = []

  with torch.no_grad():
    for batch in tqdm(valid_dl, desc='Evaluation'):
      batch = tuple(t.to(device) for t in batch)
      input_ids = batch[0]
      seq_lens = batch[1]
      labels = batch[2]

      _, logits = model(input_ids, seq_lens, labels=labels) # logits: batch_size x classes
      labels_all.extend(list(to_numpy(labels)))
      preds_all.extend(list(np.argmax(to_numpy(logits), axis=1)))

    predictions = np.asarray(preds_all).reshape(-1)
    targets = np.asarray(labels_all).reshape(-1)

    acc = accuracy(predictions, targets)
    p, r, f1, _ = precision_recall_fscore_support(targets, predictions) # F1 when beta = 1 (default)
    return acc, p, r, f1


def train(
    model: nn.Module,
    train_dl: DataLoader,
    valid_dl: DataLoader,
    optimizer: torch.optim.Optimizer,
    n_epochs: int,
    device: torch.device,
    scheduler = None,
    patience: int = 10,
    lang = 'str'
):
  losses = []
  best_acc = 0.0
  pcounter = 0

  learning_rates = []
  best_performance = (0,0,0,0)

  for ep in range(n_epochs):

    loss_epoch = []

    for batch in tqdm(train_dl):
      model.train()
      optimizer.zero_grad()

      batch = tuple(t.to(device) for t in batch)
      input_ids = batch[0]
      seq_lens = batch[1]
      labels = batch[2]

      loss, logits = model(input_ids, seq_lens, labels=labels)
      losses.append(loss.item())
      loss_epoch.append(loss.item())

      loss.backward()

      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

      optimizer.step()
      if scheduler != None:
        scheduler.step()
        learning_rates.append(scheduler.get_last_lr()[0])


    acc, p, r, f1 = evaluate(model, valid_dl)
    print(f'Validation accuracy: {acc}, train loss: {sum(loss_epoch) / len(loss_epoch)}')

    if acc > best_acc:
      torch.save(model.state_dict(), 'best_model')
      best_acc = acc
      pcounter = 0
      best_performance = (acc,p,r,f1)
    else:
      pcounter += 1
      if pcounter == patience:
        break
        #gc.collect()


  torch.save(model.state_dict(), 'final_model_weights.pth')
  model.load_state_dict(torch.load('best_model'))

  return model, losses, learning_rates, best_performance


### Create model, train and eval

#### Hyperparams

In [ ]:
# Model
num_layers = 2
dropout_rate = 0.1
hidden_dim = 256
n_classes = 2

# Training
batch_size = 64
n_epochs = 10
lr = 1e-3 # Scheduled
num_workers = 2

# not used
#tie_weights = True if embedding_dim == hidden_dim else False
# seq_len = 5
clip = 0.25
#saved = False

#### Arabic

In [ ]:
print(f"""Model params:
  layers: {num_layers}
  dropout: {dropout_rate}
  hidden_dim: {hidden_dim}
  n_classes: {n_classes}
  batch_size: {batch_size}
  n_epochs: {n_epochs}
  lr: {lr}
  num_workers: {num_workers}""")

# Create the dataset reader
train_dataset = ClassificationDatasetReader(train_arab, bpemb_ar)
valid_dataset = ClassificationDatasetReader(val_arab, bpemb_ar)

# dataset loaded lazily with N workers in parallel
train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_batch_bilstm, num_workers=num_workers)
valid_dl = DataLoader(valid_dataset, batch_size=batch_size, collate_fn=collate_batch_bilstm, num_workers=num_workers)

# Create model
model = BiLSTM(
    pretrained_embeddings = torch.FloatTensor(embeddings_ar),
    lstm_dim = hidden_dim,
    num_layers = num_layers,
    dropout_prob = dropout_rate,
    n_classes=2
  ).to(device)

# Create the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = CyclicLR(optimizer, base_lr=0., max_lr=lr, step_size_up=1, step_size_down=len(train_dl)*n_epochs, cycle_momentum=False)

# Train
model, losses, learning_rates, best_performance = train(model, train_dl, valid_dl, optimizer, n_epochs, device, scheduler, 10, 'arabic')
a, p, r, f1 = best_performance
print(f"""Best model performance:
Acc:       {a}
Precision: {p}
Recall:    {r}
f1-score:  {f1}
""")

Model params:
  layers: 2
  dropout: 0.1
  hidden_dim: 256
  n_classes: 2
  batch_size: 64
  n_epochs: 10
  lr: 0.001
  num_workers: 2


Evaluation: 100%|██████████| 30/30 [00:01<00:00, 18.72it/s]


Validation accuracy: 0.849106203995794, train loss: 0.42038984410062463


Evaluation: 100%|██████████| 30/30 [00:01<00:00, 18.62it/s]


Validation accuracy: 0.8606729758149316, train loss: 0.35409037781714364


...

Validation accuracy: 0.8843322818086226, train loss: 0.16916302179205622


Evaluation: 100%|██████████| 30/30 [00:01<00:00, 16.05it/s]


Validation accuracy: 0.8848580441640379, train loss: 0.13788818452933693


Evaluation: 100%|██████████| 30/30 [00:01<00:00, 15.01it/s]

Validation accuracy: 0.8796004206098843, train loss: 0.10955739308542875
Best model performance:
Acc:       0.8848580441640379
Precision: [0.89102564 0.87888199]
Recall:    [0.87697161 0.89274448]
f1-score:  [0.88394277 0.885759  ]



In [ ]:
def avg(x, y):
  return (x+y)/2

print(avg(0.89102564, 0.87888199))
print(avg(0.87697161, 0.89274448))
print(avg(0.88394277, 0.885759))

0.884953815
0.884858045
0.884850885


#### Bengali

In [ ]:
# Create dataloader
train_dataset = ClassificationDatasetReader(train_beng, bpemb_bn)
valid_dataset = ClassificationDatasetReader(val_beng, bpemb_bn)

# dataset loaded lazily with N workers in parallel
train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_batch_bilstm, num_workers=num_workers)
valid_dl = DataLoader(valid_dataset, batch_size=len(val_beng), collate_fn=collate_batch_bilstm, num_workers=num_workers)

# Create model
model = BiLSTM(
    pretrained_embeddings = torch.FloatTensor(embeddings_bn),
    lstm_dim = hidden_dim,
    num_layers = num_layers,
    dropout_prob = dropout_rate,
    n_classes=2
  ).to(device)

# Create optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# Train
model, losses, learning_rates, best_performance = train(model, train_dl, valid_dl, optimizer, n_epochs, device, scheduler, 10, 'bengali')
a, p, r, f1 = best_performance
print(f"""Best model performance:
Acc:       {a}
Precision: {p}
Recall:    {r}
f1-score:  {f1}
""")

Evaluation: 100%|██████████| 1/1 [00:00<00:00,  1.80it/s]


Validation accuracy: 0.7053571428571429, train loss: 0.5926741131146749


Evaluation: 100%|██████████| 1/1 [00:00<00:00,  1.99it/s]


Validation accuracy: 0.6785714285714286, train loss: 0.5324164545536041


Evaluation: 100%|██████████| 1/1 [00:00<00:00,  2.79it/s]


...

Validation accuracy: 0.75, train loss: 0.3593708393971125


Evaluation: 100%|██████████| 1/1 [00:00<00:00,  2.28it/s]


Validation accuracy: 0.7053571428571429, train loss: 0.31610697428385415


Evaluation: 100%|██████████| 1/1 [00:00<00:00,  1.97it/s]

Validation accuracy: 0.71875, train loss: 0.28953966856002805
Best model performance:
Acc:       0.75
Precision: [0.74137931 0.75925926]
Recall:    [0.76785714 0.73214286]
f1-score:  [0.75438596 0.74545455]



In [ ]:
print(avg(0.74137931, 0.75925926))
print(avg(0.76785714, 0.73214286))
print(avg(0.75438596, 0.74545455))

0.750319285
0.75
0.749920255


#### Indonesian

In [ ]:
# Create datloader
train_dataset = ClassificationDatasetReader(train_indo, bpemb_id)
valid_dataset = ClassificationDatasetReader(val_indo, bpemb_id)

# dataset loaded lazily with N workers in parallel
train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_batch_bilstm, num_workers=num_workers)
valid_dl = DataLoader(valid_dataset, batch_size=len(val_indo), collate_fn=collate_batch_bilstm, num_workers=num_workers)

# Create model
model = BiLSTM(
    pretrained_embeddings = torch.FloatTensor(embeddings_id),
    lstm_dim = hidden_dim,
    num_layers = num_layers,
    dropout_prob = dropout_rate,
    n_classes=2
  ).to(device)

# Create the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# Train
model, losses, learning_rates, best_performance = train(model, train_dl, valid_dl, optimizer, n_epochs, device, scheduler, 10, 'indonesian')
a, p, r, f1 = best_performance
print(f"""Best model performance:
Acc:       {a}
Precision: {p}
Recall:    {r}
f1-score:  {f1}
""")

Evaluation: 100%|██████████| 1/1 [00:01<00:00,  1.37s/it]


Validation accuracy: 0.7850545759865659, train loss: 0.527604009852063


Evaluation: 100%|██████████| 1/1 [00:01<00:00,  1.34s/it]


Validation accuracy: 0.7951301427371956, train loss: 0.459198495945451


Evaluation: 100%|██████████| 1/1 [00:01<00:00,  1.36s/it]


...

Validation accuracy: 0.7808564231738035, train loss: 0.2921182501166226


Evaluation: 100%|██████████| 1/1 [00:01<00:00,  1.39s/it]


Validation accuracy: 0.7858942065491183, train loss: 0.2552653907449635


Evaluation: 100%|██████████| 1/1 [00:01<00:00,  1.13s/it]

Validation accuracy: 0.7732997481108312, train loss: 0.21925048521777105
Best model performance:
Acc:       0.8077246011754828
Precision: [0.77526395 0.84848485]
Recall:    [0.86531987 0.75041876]
f1-score:  [0.81782021 0.79644444]



In [ ]:
print(avg(0.77526395, 0.84848485))
print(avg(0.86531987, 0.75041876))
print(avg(0.81782021, 0.79644444))

0.8118744
0.807869315
0.807132325


### Results